In [1]:
import nltk,os
nltk.download('gutenberg')

from nltk.corpus import gutenberg
import pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')

os.makedirs('data',exist_ok=True)

with open('data/hamlet.txt','w') as file:
    file.write(data)
    
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package gutenberg to C:\Users\Rishad
[nltk_data]     R\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
with open('data/hamlet.txt','r') as file:
    text=file.read().lower()
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(total_words)

4818


In [4]:
inputsequences=[]

for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequences=token_list[:i+1]
        inputsequences.append(n_gram_sequences)
        
max_len=max([len(x) for x in inputsequences])
max_len

14

In [5]:
input_sequences=np.array(pad_sequences(inputsequences,maxlen=max_len,padding='pre'))
input_sequences

x,y=input_sequences[:,:-1],input_sequences[:,-1]
import tensorflow as tf

y=tf.keras.utils.to_categorical(y,num_classes=total_words)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout,GRU

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(100))
model.add(Dropout(0.4))
model.add(Dense(total_words,activation='softmax'))

In [6]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

history=model.fit(x_train,y_train,
                  epochs=50,
                  validation_data=(x_test,y_test),
                  verbose=1,
                  )


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 gru (GRU)                   (None, 13, 150)           113400    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 gru_1 (GRU)                 (None, 100)               75600     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1157418 (4.42 MB)
Trainable params: 11574

In [7]:
def predict_next_word(model,tokenizer,text,max_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_len:
        token_list=token_list[-(max_len-1):]
        
    token_list=pad_sequences([token_list],maxlen=max_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [8]:
input_text="To be or not to be "
print(f'this the input text --{input_text}')
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f'next word is --{next_word}')

this the input text --To be or not to be 
next word is --wish'd


In [9]:
model.save('model/GRU.h5')

e:\machine learning\project\predict next word\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
